In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader

import numpy as np

In [2]:
labels = np.load("labels.npy")

training_generator = DataLoader(labels, 128)

In [3]:
model = Sequential()
model.add(Dense(768, activation='relu', input_shape=(98304,)))
# model.add(Dense(768, activation='relu'))
model.add(Dense(768, activation='sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2022-02-26 17:46:28.931752: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-26 17:46:28.943814: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-26 17:46:28.944177: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-26 17:46:28.945392: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
model.fit(training_generator, epochs=1)

2022-02-26 17:46:32.936346: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-02-26 17:46:33.294486: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-02-26 17:46:33.405897: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.


    5/10457 [..............................] - ETA: 12:36 - loss: 6.6133 - accuracy: 0.0156    

2022-02-26 17:46:33.608620: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.
2022-02-26 17:46:33.696658: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50331648 exceeds 10% of free system memory.


 1952/10457 [====>.........................] - ETA: 12:07 - loss: 5.1845 - accuracy: 0.0750

In [7]:
import time
model.save("models/model-{}".format(int(time.time())))

INFO:tensorflow:Assets written to: models/model-1645905292/assets
